In [1]:
import dataset_utils
import os
import random
import torch
import matplotlib.pyplot as plt
import numpy as np
from config import cfg
from datasets import LandCoverDataset
from models.deeplab import get_model as get_deeplab_model
from PIL import Image
from torch.utils.data import DataLoader, Subset
import albumentations as album
from utils import get_validation_augmentation, get_training_augmentation, get_preprocessing, save_history, save_model, reverse_one_hot, colour_code_segmentation, visualize

In [2]:
LANDCOVER_ROOT = '/root/deepglobe'
cfg.merge_from_file('cfg/deeplab_resnet50_advance_aug.yaml')

In [3]:
image_path, mask_path = '/root/deepglobe/train/10452_sat.jpg', '/root/deepglobe/train/10452_mask.jpg'

In [4]:
train_df, val_df = dataset_utils.get_landcover_train_val_df(LANDCOVER_ROOT, random_state=cfg.SEED)
dataset_info = dataset_utils.get_landcover_info(LANDCOVER_ROOT, include_unknow=False)
class_names = dataset_info['class_names']
class_rgb_values = dataset_info['class_rgb_values']
select_class_rgb_values = dataset_info['select_class_rgb_values']

In [6]:
# augmentations_dict = {
#     'HueSaturationValue': album.HueSaturationValue(p=1),
#     'RandomBrightnessContrast': album.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, always_apply=True),
#     'Blur': album.Blur(blur_limit=7, always_apply=True),
#     'ToGray': album.ToGray(always_apply=True),
#     'IAASharpen': album.IAASharpen(always_apply=True),
#     'RGBShift': album.RGBShift(always_apply=True),
#     'GaussNoise': album.GaussNoise(always_apply=True),
#     'RandomGamma': album.RandomGamma(always_apply=True)
# } 

level1_augs = [album.HueSaturationValue(p=1),
               album.Blur(blur_limit=7, always_apply=True),
               album.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, always_apply=True)]
level2_augs = [album.GaussNoise(always_apply=True),
               album.IAASharpen(always_apply=True),
               album.RGBShift(always_apply=True)]
level3_augs = [album.RandomGamma(always_apply=True),
               album.ToGray(always_apply=True)]

augmentations_dict = {
    'level1': level1_augs,
    'level2': level1_augs + level2_augs,
    'level3': level1_augs + level2_augs + level3_augs,
}

In [7]:
image_path = '/root/deepglobe/train/10452_sat.jpg'
sample_dir = 'augment_functions_sample'

In [8]:
image = Image.open(image_path)
image_np = np.array(image)

In [11]:
for aug_name, aug_func_list in augmentations_dict.items():
    aug_func = album.Compose(aug_func_list)
    augmented_sample = aug_func(image=image_np)
    augmented_image_np = augmented_sample['image']
    Image.fromarray(augmented_image_np).save(os.path.join(sample_dir, f'{aug_name}.jpeg'))